In [1]:
import numpy as np
import random
import math

In [2]:
def LineGenerate(lenght, m):
    """
    Creating a line with m random segments
    """
    line = np.zeros(lenght,)
    ab = np.array(sorted(random.sample(range(0, lenght-1), m*2))) # random start and end of the segments
    ab = ab.reshape((m,2)) # start and end of the segments coordinates
    # positive points coordinates
    ones = np.arange(ab[0,0],ab[0,1])
    for i in range(1,m):
        ones = np.concatenate((ones, np.arange(ab[i,0],ab[i,1])))
    line[ones] = 1  # put the generated segments on the line
    return line, ones, ab

In [3]:
def FindAssociated(c, ab, base, L):
    # distance from the farthest point to the base
    b = math.sqrt(c**2+base[1]**2)
    # distance from the farthest point to greedy
    dist = (L**2-2*L*b)/(2*(L-b-c))
    c_ass = c-dist
    for i in range(0,ab.shape[0]):
        if (c_ass > ab[i-1,1]) & (c_ass < ab[i,0]):
            c_ass = ab[i,0]
        if (c_ass < ab[0,0]):
            c_ass = ab[0,0]  
    return c_ass


In [4]:
def Candidates(base, L, ab):
    on_gap = False
    xB = ab[ab.shape[0]-1,1]
    c = np.array(xB)
    while not on_gap:
        # distance from the farthest point to the base
        b = math.sqrt(xB**2+base[1]**2)
        # distance from the farthest point to greedy
        dist = (L**2-2*L*b)/(2*(L-b-xB))
        # greedy point coordinate
        xA = xB-dist
        c = np.append(c, xA)
        xB = xA
        for i in range(0,ab.shape[0]):
            if (xA > ab[i-1,1]) & (xA < ab[i,0]):
                on_gap = True 
            if (xA < ab[0,0]):
                on_gap = True 
    c = c[:c.size-1]
    return c

In [5]:
def AllCandidates(base, L, ab):
    ab_cut = ab
    c = np.empty(0)
    while not (ab_cut.size == 0):
        c_i = Candidates(base, L, ab_cut)
        c = np.append(c, c_i)       
        ab_cut = ab_cut[:ab_cut.shape[0]-1]
    return c

In [6]:
def SegmentNumber(c, ab):
    for i in range(0,ab.shape[0]):
        if (c >= ab[i,0]) & (c <= ab[i,1]):
            return i  

In [7]:
def TourLenght(point1, point2, base):
    return max(point1, point2) - min(point1, point2) + math.sqrt(point1**2+base[1]**2) + math.sqrt(point2**2+base[1]**2)

In [8]:
def SeparateLeftRight(ab, base):
    ab_right = ab[np.argmax(ab[:,0] > base[0]):] - base[0]
    ab_left = np.flip(base[0] - ab[:np.argmax(ab[:,1] > base[0])])
    for i in range(0,ab.shape[0]):
        if (base[0] > ab[i,0]) & (base[0] < ab[i,1]):
            central_right = np.array([[base[0], ab[i,1]]])-base[0]
            central_left = -(np.array([[base[0],ab[i,0]]])-base[0])
            ab_right = np.append(central_right, ab_right, axis=0)
            ab_left = np.append(central_left, ab_left, axis=0)
    return ab_left, ab_right

In [9]:
def FindCentralTours(c_left, c_right , base, L):
    j = 0
    central_tours = np.empty((0,3))
    tour_lenght = 0
    while (tour_lenght <= L) and (j < c_left.size):
        central_tours_1 = np.empty((0,3))
        i = 0
        while (tour_lenght <= L) and (i < c_right.size):
            dist_left = math.sqrt(c_left[j]**2+base[1]**2)
            dist_right = math.sqrt(c_right[i]**2+base[1]**2)
            tour_lenght = c_left[j] + c_right[i] + dist_left + dist_right
            tour = np.array((c_left[j], c_right[i], tour_lenght)).reshape(1,3)
            central_tours_1 = np.append(central_tours_1, tour, axis=0)
            i += 1
        central_tours = np.append(central_tours, central_tours_1[central_tours_1[:,2]<=L,:], axis=0)
        tour_lenght = 0
        j += 1
    return central_tours

In [23]:
def DPwithPoints(c, ab, base, L):   
    c = np.sort(c)  
    E = np.zeros(c.size) # check if zeros are ok
    Start_Points = np.zeros((c.size,c.size)) # check if zeros are ok
    End_Points = np.zeros((c.size,c.size))

    for k in range(0, c.size):
        c_ass = FindAssociated(c[k], ab, base, L)
        
        j = np.arange(SegmentNumber(c_ass, ab), SegmentNumber(c[k], ab)+1)  
    
        if c_ass == ab[0,0]:
            E[k] = TourLenght(c[k], ab[0,0], base)  
            
            Start_Points[k,0] = c[k]
            End_Points[k,0] = ab[0,0]
            
        elif c_ass in ab[1:,0]:      
            
            func = np.empty(0)
            for i in j:
                func = np.append(func, TourLenght(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])
            E[k] = min(func)
            index_min = np.argmin(func)+j[0]
            
            Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
            End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                                   
            Start_Points[k, np.argmax(Start_Points[k,:]==0)] = c[k]
            End_Points[k, np.argmax(End_Points[k,:]==0)] = ab[index_min,0]
    
        else:             
            func = np.empty(0)
            if (c_ass < ab[SegmentNumber(c[k], ab),0]): 
                for i in j[1:]:
                    func = np.append(func, TourLenght(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])  
            else:
                func = [1000000.0]
            func2 = L + E[c.tolist().index(c_ass)]
            
            if (func2>min(func)):
                index_min = np.argmin(func)+j[0]
                Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,np.argmax(End_Points[k,:]==0)] = ab[index_min,0]
            else:
                Start_Points[k,:] = Start_Points[c.tolist().index(c_ass),:] 
                End_Points[k,np.argmax(End_Points[k,:]==0)] = c_ass
                
            Start_Points[k,np.argmax(Start_Points[k,:]==0)] = c[k]
                                 
            E[k] = min(func2, min(func))
            
    Start_Points = Start_Points[c.size-1]
    End_Points = End_Points[c.size-1]
    return E[c.size-1], Start_Points[Start_Points>0], End_Points[End_Points>0]

In [11]:
def OneSideWithPoints(c, c_first, ab):
    E = np.zeros(c_first.size)
    StartPoints = np.zeros((c_first.size,c.size))
    EndPoints = np.zeros((c_first.size,c.size))
    ab_i = ab.astype(np.float64)
    for i in range(0, c_first.size):
        if c_first[i] != ab_i[0,1]:
            ab_i[0,0] = c_first[i]
        else:
            ab_i = ab[ab[:,0]>c_first[i]].astype(np.float64)
        if (c[c>c_first[i]].size != 0):
            E[i],  Start_Points, End_Points = DPwithPoints(c[c>c_first[i]], ab_i, base, L) 
            StartPoints[i,:Start_Points.size] = Start_Points
            EndPoints[i,:End_Points.size] = End_Points
            
        else: E[i] = 0
    return E, StartPoints, EndPoints

In [65]:
def JoinTours(StartPoints, EndPoints):
    Start = np.array([StartPoints[StartPoints>0]])
    End = np.array([EndPoints[EndPoints>0]])
    
    return np.append(Start, End, axis = 0)

In [108]:
def BothSidesWithPoints(ab, base, L):
    ab_left, ab_right = SeparateLeftRight(ab, base)
    
    c_right = np.sort(AllCandidates(base, L, ab_right))
    c_left =  np.sort(AllCandidates(base, L, ab_left))
    
    central_tours = FindCentralTours(c_left, c_right , base, L)
    
    c_left_first = np.unique(np.sort(central_tours[:,0]))
    c_right_first = np.unique(np.sort(central_tours[:,1]))
    
    E_left, StartPoints_left, EndPoints_left = OneSideWithPoints(c_left, c_left_first, ab_left)
    E_right, StartPoints_right, EndPoints_right = OneSideWithPoints(c_right, c_right_first, ab_right)
        
    left_central = np.append(c_left_first.reshape(E_left.shape[0],1), E_left.reshape(E_left.shape[0],1), axis=1)
    right_central = np.append(c_right_first.reshape(E_right.shape[0],1), E_right.reshape(E_right.shape[0],1), axis=1)
      
    totalL = np.zeros(central_tours.shape[0])
    for i in range (0, central_tours.shape[0]):
        a = tuple(np.argwhere(left_central == central_tours[i,0])[0])
        b = tuple(np.argwhere(right_central == central_tours[i,1])[0])
        totalL[i] = left_central[a[0],1] + right_central[b[0],1] + float(central_tours[i,2])
        
    c = tuple(np.argwhere(left_central == central_tours[np.argmin(totalL),0])[0])
    d = tuple(np.argwhere(right_central == central_tours[np.argmin(totalL),1])[0])
              

    E_left_max, StartPoints_left_max, EndPoints_left_max = DPwithPoints(c_left, ab_left, base, L)
    E_right_max, StartPoints_right_max, EndPoints_right_max = DPwithPoints(c_right, ab_right, base, L)
    
    if (E_left_max + E_right_max) < min(totalL):
        Tour_right = JoinTours(StartPoints_right_max, EndPoints_right_max) + base[0]
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = base[0] - JoinTours(StartPoints_left_max, EndPoints_left_max)
        Tour = np.append(Tour_left, Tour_right, axis = 1)
        
    else:       
        Tour_right = JoinTours(StartPoints_right[d[0]], EndPoints_right[d[0]]) + base[0]
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = base[0] - JoinTours(StartPoints_left[c[0]], EndPoints_left[c[0]])
        Tour_central = np.array([[base[0] - central_tours[np.argmin(totalL),0], central_tours[np.argmin(totalL),1] + base[0]]]).reshape((2, 1))
        
        Tour = np.append(Tour_left, Tour_central, axis = 1)
        Tour = np.append(Tour, Tour_right, axis = 1)
        
    TotalLenght = min((E_left_max + E_right_max), min(totalL)) 
    
    for i in range(Tour.shape[1]):
        print('Tour', i+1, 'start:', Tour[0,i])
        print('Tour', i+1, 'end:', Tour[1,i])
        print('Tour', i+1, 'lenght:', TourLenght(abs(Tour[0,i]-base[0]), abs(Tour[1,i]-base[0]), base))
    print('Total number of tours:', Tour.shape[1])
    print('Total lenght:', TotalLenght)
    
    return Tour, TotalLenght

In [13]:
def FindTheFarthest(base,ones,first,last):
    """
    Looking for the farthest point from the base
    """
    # distances to the base station
    fdist = math.sqrt((first-base[0])**2+base[1]**2)
    ldist = math.sqrt((last-base[0])**2+base[1]**2)

    # comparing the distances
    if fdist > ldist:
        farthest = first
        dist = fdist
    else:
        farthest = last
        dist = ldist
    
    return farthest, dist

In [14]:
def GreedyAB(base,line, L, ab):
    covered = False
    left = ab[0,0] # the leftmost point of the first segment
    right = ab[ab.shape[0]-1,1] # the rightmost point of the last segment
    farthest, far_dist = FindTheFarthest(base,ones,left,right) # the farthest point from the base
    if (L < far_dist*2):
        print ("we can not achieve the farthest point")
    else:
        n = 0 # number of tours
        totalL = 0 # total length
        ab_short = ab
        
        while not covered:
            left = ab_short[0,0] # the leftmost point of the first segment
            right = ab_short[ab_short.shape[0]-1,1] # the rightmost point of the last segment
            farthest, far_dist = FindTheFarthest(base,ones,left,right)
            # the farthest point coordinate
            xB = abs(farthest-base[0])
            # distance from the farthest point to the base
            b = far_dist
            # distance from the farthest point to greedy
            c = (L**2-2*L*b)/(2*(L-b-xB))
            
            if c >= (ab_short[ab_short.shape[0]-1,1]-ab_short[0,0]):
                tour_start = ab_short[0,0]
                tour_end = ab_short[ab_short.shape[0]-1,1]
                covered = True
                
            else:
                tour_start = farthest
                if (farthest == left):
                    if (c >= xB):
                        ab_short = ab_short[ab_short[:,1] > base[0]] 
                        ab_short[0,0] = base[0] 
                        tour_end = base[0]
                    else:
                        ab_left = np.flip(base[0] - ab_short[:np.argmax(ab_short[:,1] > base[0])])
                        end_left = FindAssociated(xB, ab_left, base, L)
                        tour_end = base[0] - end_left
                        ab_short = ab_short[ab_short[:,1] > tour_end] 
                        if (ab_short[0,0] < tour_end):
                            ab_short[0,0] = tour_end 
                            
                elif (farthest == right):
                    if (c >= xB):
                        ab_short = ab_short[ab_short[:,0] < base[0]] 
                        ab_short[ab_short.shape[0]-1,1] = base[0] 
                        tour_end = base[0]
                        
                    else:
                        ab_right = ab_short[np.argmax(ab_short[:,0] > base[0]):] - base[0]
                        end_right = FindAssociated(xB, ab_right, base, L)
                        tour_end = end_right + base[0] 
                        ab_short = ab_short[ab_short[:,0] < tour_end]
                        if (ab_short[ab_short.shape[0]-1,1] > tour_end):
                            ab_short[ab_short.shape[0]-1,1] = tour_end
            
            
            totalL += TourLenght(tour_start-base[0], tour_end-base[0], base)
            n += 1
            print('Tour', n ,'lenght:', TourLenght(abs(tour_start-base[0]), abs(tour_end-base[0]), base))
            print('Tour', n ,'start:', tour_start)
            print('Tour', n ,'end:', tour_end)   
            
        print('Total number of tours:', n)
        print('Total lenght:', totalL)

In [15]:
line, ones, ab = LineGenerate(350, 15)
base = np.array((100,500)) # base station coordinates
L = 1120

In [109]:
Tour, TotalLenght = BothSidesWithPoints(ab, base, L)

Tour 1 start: 5.0
Tour 1 end: 71.0
Tour 1 lenght: 1075.785281110073
Tour 2 start: 82.0
Tour 2 end: 173.0
Tour 2 lenght: 1060.6247955463423
Tour 3 start: 210.0
Tour 3 end: 283.0
Tour 3 lenght: 1117.3938801624588
Tour 4 start: 309.0
Tour 4 end: 331.0
Tour 4 lenght: 1114.7055974236796
Total number of tours: 4
Total lenght: 4404.509554242553


In [17]:
GreedyAB(base, line, L, ab)

Tour 1 lenght: 1114.7055974236796
Tour 1 start: 331
Tour 1 end: 309
Tour 2 lenght: 1117.3938801624588
Tour 2 start: 283
Tour 2 end: 210
Tour 3 lenght: 1103.9449872039218
Tour 3 start: 5
Tour 3 end: 100
Tour 4 lenght: 1078.3009004543728
Tour 4 start: 100
Tour 4 end: 173
Total number of tours: 4
Total lenght: 4414.345365244433


In [18]:
ab

array([[  5,  11],
       [ 12,  28],
       [ 36,  43],
       [ 59,  71],
       [ 82,  83],
       [ 85,  90],
       [ 99, 104],
       [111, 113],
       [116, 117],
       [134, 173],
       [210, 229],
       [250, 254],
       [274, 283],
       [309, 312],
       [329, 331]])